In [7]:
!pip install requests
!pip install m3u8
!pip install pathlib

In [8]:
import requests
import m3u8
from urllib.parse import urlparse
import time
import pathlib
from pathlib import Path

In [9]:
def get_segment(url):
    m3u8_segments = m3u8.load(url)
    segment = m3u8_segments.data["segments"][0]
    return segment

In [10]:
def get_stream(url, target_length = 20):
    urltemp = url.split("/")
    base_url = ""
    for i in range(len(urltemp)-1):
        base_url = base_url + urltemp[i] + "/"

    length = 0

    file_name_temp = urltemp[-2].split(".")

    temp_url = ""

    # frame_count = 0
    # model = load_saved_model()

    current_time = time.strftime("%Y_%m_%d_%H-%M-%S", time.localtime())

    while(length<target_length):
        segment = get_segment(url)
        segment_url = base_url + segment["uri"]

        if(segment_url != temp_url):
            chunk_count=0
            request = requests.get(segment_url, stream=True)
            if(request.status_code == 200):
                # file_save_path = save_path.__str__() + "_" + str(i) + ".mp4" 
                file_name = file_name_temp[0] + "_" + str(current_time)

                save_path = Path.cwd() / file_name
                file_save_path = save_path.__str__() + ".mp4" 
                print(file_save_path)
                with open(file_save_path,'ab') as file:
                    for chunk in request.iter_content(chunk_size=1024):
                        chunk_count += 1
                        # print(chunk_count)
                        file.write(chunk)
                        # if chunk_count>1024:
                        #     print('File Too Big (Greater than 1MB per .ts segment. Error Suspected. Ending.')
                        #     break
                    print("here")
                file.close()
                # image_path, frame_count = image_extraction_stream(file_save_path, duration=segment["duration"])
                # run_model_on_file(model, image_path)
                # print("Frame Count =", frame_count)
                length = length + segment["duration"]
                print("Video Length: ", length)
            else:
                print("ERROR", request.status_code)
        temp_url = segment_url
        
        
        # print("Frame Count =", frame_count)
        
    
    # image_extraction(file_save_path)

    # print(total_video_frames)

In [12]:
get_stream("https://wzmedia.dot.ca.gov/D5/1atStateParkDr.stream/chunklist_w64945119.m3u8", target_length=300)

URLError: <urlopen error [Errno 2] No such file or directory>